In [4]:
import sys
from pathlib import Path

# Add project root to Python path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [3]:
from azure.identity.aio import AzureCliCredential
from agent_framework.azure import AzureAIClient
from src.modules import ViewSelectorModule
from src.data_loader import load_snowflake_views
import dspy

In [9]:
# Cell 3: Configure DSPy and load data
lm = dspy.LM("azure/gpt-4o")
dspy.configure(lm=lm)

In [5]:
import json

with open("../data/snowflake_view.json", 'r') as f:
    data = json.load(f)
    snowflake_views = data.get('views', [])
    print(f"✅ Loaded {len(snowflake_views)} views directly")

✅ Loaded 21 views directly


In [ ]:
# snowflake_views = load_snowflake_views("../data/snowflake_view.json")

In [38]:
from src.modules_v2 import IntegratedText2SQLModule
view_selector = IntegratedText2SQLModule(config = {},candidate_views=snowflake_views)

In [8]:
# # Create a fresh module
# view_selector = ViewSelectorModule(candidate_views=snowflake_views)
# # Load the optimized state
# view_selector.load("../data/optimized_modules/gepa_module.json")

In [46]:
prediction = view_selector(
        question="What is MICs current exposure in the GCC in Private Equity platform?",
        conversation_history=[]
        )
print(prediction)

Prediction(
    entity_reasoning='1. **Identify terms in query that match entity candidates:**\n   - The query mentions "Private Equity" and "platform".\n   - "Private Equity" is a term that appears in both PLATFORM and ASSET_CLASS categories.\n\n2. **Apply disambiguation rules based on context keywords:**\n   - The query includes the word "platform", which suggests that we should prefer the PLATFORM entity type over ASSET_CLASS.\n\n3. **Match terms to EXACT values from entity_mapper.json:**\n   - "Private Equity" matches exactly with a value in the PLATFORM category: "Private Equity".\n\n4. **Validate: no partial matches, no JSON artifacts, proper capitalization:**\n   - The matched term "Private Equity" is an exact match with the correct capitalization as listed in the PLATFORM category.\n   - No partial matches or JSON artifacts are present in the resolved entities.',
    resolved_entities='{"PLATFORM": "Private Equity"}',
    view_reasoning='1. **Analyze question keywords and resol

In [39]:
def select_relevant_views(question: str, conversation_history: str = "") -> dict:
    """
    Select relevant database views for a given question.
    
    Args:
        question: User's natural language database query
        conversation_history: Previous conversation context (optional)
    
    Returns:
        Dictionary with selected views and reasoning
    """
    prediction = view_selector(
        question=question,
        conversation_history=conversation_history
    )
    
    return {
        "selected_views": prediction.selected_views,
        "reasoning": prediction.reasoning
    }

In [23]:
def get_entity_schema(entity_name: str) -> str:
    """
    Get the schema information for a specific database view entity.
    
    Args:
        entity_name: The entity name (e.g., 'INVESTMENT_KPI_VW', 'MIC_BY_COUNTRY_VW')
    
    Returns:
        String with schema details including columns, types, and definitions
    """
    # Find the view by entity name
    view_info = None
    view_display_name = None
    
    for view in snowflake_views:
        if view.get('entity', '') == entity_name:
            view_info = view
            view_display_name = view.get('view_name', entity_name)
            break
    
    if not view_info:
        # List available entities for debugging
        available_entities = [v.get('entity', '') for v in snowflake_views if v.get('entity')][:10]
        return f"❌ Entity '{entity_name}' not found.\n\nAvailable entities: {', '.join(available_entities)}..."
    
    # Build schema information
    schema_info = f"""**Schema for {view_display_name}**
**Entity:** {entity_name}
**Description:** {view_info.get('description', 'N/A')}

**Columns:**
"""
    
    if 'columns' in view_info and view_info['columns']:
        for col in view_info['columns'][:20]:  # Limit to first 20 columns
            col_name = col.get('name', 'Unknown')
            col_type = col.get('type', 'Unknown')
            col_def = col.get('definition', '')
            
            schema_info += f"\n• **{col_name}** ({col_type})"
            if col_def:
                schema_info += f"\n  {col_def}"
            
            # Add allowed values if present
            if col.get('allowed_values'):
                values = col['allowed_values'][:5]  # First 5 values
                schema_info += f"\n  Allowed values: {', '.join(map(str, values))}"
                if len(col['allowed_values']) > 5:
                    schema_info += f" (+ {len(col['allowed_values']) - 5} more)"
            
            schema_info += "\n"
        
        if len(view_info['columns']) > 20:
            schema_info += f"\n... and {len(view_info['columns']) - 20} more columns"
    else:
        schema_info += "No column information available"
    
    return schema_info


print("✅ Tool functions created: select_relevant_views, get_entity_schema")

✅ Tool functions created: select_relevant_views, get_entity_schema


### Example of running by MAF with 2 steps: 
- Select correct views
- 

In [42]:
async def non_streaming_example() -> None:
    """Example of non-streaming response (get the complete result at once)."""
    print("=== Non-streaming Response Example ===")

    # Since no Agent ID is provided, the agent will be automatically created.
    # For authentication, run `az login` command in terminal or replace AzureCliCredential with preferred
    # authentication option.
    async with (
        AzureCliCredential() as credential,
        AzureAIClient(credential=credential).create_agent(
            name="BasicWeatherAgent",
            instructions='''You are an expert Text-to-SQL agent for Snowflake databases.
                Your workflow:
                1. **Understand the question**: Identify what data the user needs
                2. **Select views**: Use select_relevant_views to find which database views contain the needed data
                3. **Get schema**: Use get_entity_schema for each selected view to understand the exact column names and types
                4. **Generate SQL**: Write a Snowflake SQL query using the schema information

                Important rules:
                - Always call select_relevant_views first to identify relevant views. Display the output of select_relevant_views.
                - Then call get_entity_schema for each selected view to see available columns
                - Use exact column names from the schema (case-sensitive)
                - Generate valid Snowflake SQL syntax
                - Explain your reasoning at each step''',
            tools=[select_relevant_views, get_entity_schema],
        ) as agent,
    ):
        query = "What is MICs current exposure in the GCC in Private Equity platform?"
        print(f"User: {query}")
        async for chunk in agent.run_stream(query):
            if chunk.text:
                print(chunk.text, end="", flush=True)
        print("\n")

In [43]:
result = await non_streaming_example()
print(result)

=== Non-streaming Response Example ===
User: What is MICs current exposure in the GCC in Private Equity platform?
Based on the query and schema details, we need to identify MIC's current exposure in the GCC for the Private Equity platform. The relevant columns from the `PLATFORM_BY_REGION_VW` view are:

1. **PLATFORM**: This will filter for "Private Equity."
2. **REGION**: This will filter for GCC countries under "Middle East & North Africa."
3. **UNREALIZED_VALUE**: Represents the exposure or fair market value.

We will also retrieve the most recent data available, so we'll use the **AS_ON_DATE_ID** column to filter for the latest date.

### Query
Here is the SQL query:

```sql
SELECT 
    AS_ON_DATE_ID,
    PLATFORM,
    REGION,
    UNREALIZED_VALUE
FROM 
    PLATFORM_BY_REGION_VW
WHERE 
    PLATFORM = 'Private Equity' 
    AND REGION = 'Middle East & North Africa'
    AND AS_ON_DATE_ID = (
        SELECT MAX(AS_ON_DATE_ID)
        FROM PLATFORM_BY_REGION_VW
        WHERE PLATFORM = 

In [45]:
result = await non_streaming_example()
print(result)

=== Non-streaming Response Example ===
User: What is MICs current exposure in the GCC in Private Equity platform?
### Analysis of Query and Schema

1. **Question Breakdown**:
   - The user is asking for MIC's current exposure.
   - "GCC" is a geographical location, and private equity is the investment platform.
   - "Current exposure" aligns closely with the `UNREALIZED_VALUE`, which represents fair market value or exposure of the investment at a specific point in time.

2. **Schema Review**:
   - We need data aggregated by `PLATFORM` (Private Equity) and `REGION` (GCC). The schema of `PLATFORM_BY_REGION_VW` contains both these fields.
   - `AS_ON_DATE_ID` will be used to get the most recent exposure data (the latest available date).
   - Only rows where `PLATFORM = 'Private Equity'` and `REGION = 'Middle East & North Africa'` (to account for GCC) are relevant.

### SQL Query
Below is the SQL query that retrieves the current exposure for MIC in the GCC region under the Private Equity p

## Extended Architecture: Multi-Task DSPy + Agent Framework

This notebook demonstrates how to integrate multiple specialized tasks:
1. **View Selection** - IntegratedText2SQLModule (Entity Resolution + View Selection)
2. **Schema Retrieval** - Get detailed column information
3. **SQL Generation** - Generate optimized Snowflake queries
4. **Answer Generation** - Interpret results and provide final answer

**Architecture Pattern:**
- **Agent Framework**: Orchestrates the workflow, manages conversation, coordinates tools
- **DSPy Modules**: Handle heavy reasoning for each specialized task

In [ ]:
# ============================================================================
# TASK 1: View Selection with Entity Resolution (Already implemented above)
# ============================================================================

def select_views_with_entities(question: str, conversation_history: str = "") -> dict:
    """
    Intelligent view selection using two-stage DSPy reasoning.
    
    This is the heavy reasoning task that:
    1. Resolves business entities (e.g., 'PE' → 'Private Equity')
    2. Selects optimal database views based on resolved entities
    
    Returns complete reasoning trace for transparency.
    """
    prediction = view_selector(
        question=question,
        conversation_history=conversation_history
    )
    
    return {
        "resolved_entities": prediction.resolved_entities,
        "selected_views": prediction.selected_views,
        "entity_reasoning": prediction.entity_reasoning,
        "view_reasoning": prediction.view_reasoning,
        "combined_reasoning": prediction.combined_reasoning
    }

print("✅ Task 1: View Selection (with Entity Resolution)")

In [ ]:
# ============================================================================
# TASK 2: Schema Retrieval (Already implemented above as get_entity_schema)
# ============================================================================

# Already defined above - this retrieves detailed column information
# for selected views

print("✅ Task 2: Schema Retrieval (get_entity_schema)")

In [ ]:
# ============================================================================
# TASK 3: SQL Query Generation (DSPy Module for Heavy Reasoning)
# ============================================================================

# Define SQL Writer Signature
class SQLWriterSignature(dspy.Signature):
    """
    Generate optimized Snowflake SQL queries based on question, schema, and resolved entities.
    
    You are a Snowflake SQL expert. Generate syntactically correct, optimized SQL queries.
    """
    
    question: str = dspy.InputField(desc="User's natural language question")
    view_schemas: str = dspy.InputField(desc="Detailed schema information for selected views")
    resolved_entities: str = dspy.InputField(desc="Resolved business entities (JSON format)")
    
    reasoning: str = dspy.OutputField(desc="Step-by-step SQL generation reasoning")
    sql_query: str = dspy.OutputField(desc="Complete Snowflake SQL query")


class SQLWriterModule(dspy.Module):
    """DSPy module for SQL query generation with Chain-of-Thought reasoning."""
    
    def __init__(self):
        super().__init__()
        self.sql_writer_cot = dspy.ChainOfThought(SQLWriterSignature)
    
    def forward(self, question: str, view_schemas: str, resolved_entities: str):
        """Generate SQL query with reasoning."""
        result = self.sql_writer_cot(
            question=question,
            view_schemas=view_schemas,
            resolved_entities=resolved_entities
        )
        return result


# Initialize SQL Writer
sql_writer_module = SQLWriterModule()


def generate_sql_query(question: str, view_schemas: str, resolved_entities: str) -> dict:
    """
    Generate Snowflake SQL query using DSPy Chain-of-Thought reasoning.
    
    Args:
        question: User's natural language question
        view_schemas: Detailed schema information from get_entity_schema
        resolved_entities: Business entities in JSON format
    
    Returns:
        Dict with SQL query and reasoning
    """
    prediction = sql_writer_module(
        question=question,
        view_schemas=view_schemas,
        resolved_entities=resolved_entities
    )
    
    return {
        "sql_query": prediction.sql_query,
        "reasoning": prediction.reasoning
    }


print("✅ Task 3: SQL Query Generation (DSPy Module)")

In [ ]:
# ============================================================================
# TASK 4: Answer Generation (Lightweight Agent - Fast Response)
# ============================================================================

def generate_final_answer(question: str, sql_query: str, query_results: str, resolved_entities: str) -> str:
    """
    Generate business-friendly final answer from query results.
    
    This uses the Agent Framework's native capabilities (fast, no DSPy overhead).
    For complex interpretations requiring heavy reasoning, consider using DSPy.
    
    Args:
        question: Original user question
        sql_query: SQL query that was executed
        query_results: Results from query execution
        resolved_entities: Business entities (JSON format)
    
    Returns:
        String with final answer formatted for the agent to present
    """
    # Return a structured prompt for the agent to interpret
    # Agent Framework will handle the actual generation
    context = f"""
**Question:** {question}

**Resolved Business Entities:**
{resolved_entities}

**SQL Query Executed:**
```sql
{sql_query}
```

**Query Results:**
{query_results}

**Your Task:**
Based on the query results above, provide a clear, business-friendly answer to the user's question.
Include:
1. Direct answer to the question with specific numbers
2. Context about what the data represents
3. Any relevant insights or caveats

Be concise and professional.
"""
    return context


print("✅ Task 4: Answer Generation (Lightweight Agent)")
print("   Using Agent Framework for fast response")


In [ ]:
# ============================================================================
# TASK 5: SQL Query Execution (Simple utility - not DSPy)
# ============================================================================

def execute_sql_query(sql_query: str) -> str:
    """
    Execute SQL query against Snowflake database.
    
    Note: This is a placeholder. In production, replace with actual Snowflake connection.
    
    Args:
        sql_query: Snowflake SQL query to execute
    
    Returns:
        Query results as formatted string (JSON or table)
    """
    # Placeholder implementation
    # In production, use snowflake.connector or similar
    
    # For demo purposes, return mock results
    mock_results = """
    Query Results:
    ┌─────────────────────┬───────────────────┐
    │ COUNTRY             │ UNREALIZED_VALUE  │
    ├─────────────────────┼───────────────────┤
    │ USA                 │ $2,450,000,000    │
    └─────────────────────┴───────────────────┘
    
    (1 row returned)
    """
    
    return mock_results


print("✅ Task 5: SQL Query Execution (Utility Function)")

## Complete Multi-Task Workflow with Agent Framework

Now let's integrate all tasks into a complete workflow orchestrated by Microsoft Agent Framework.

In [ ]:
async def complete_text2sql_workflow() -> None:
    """
    Complete Text-to-SQL workflow orchestrated by Microsoft Agent Framework.
    
    Architecture:
    - Agent Framework: Orchestrates the multi-step workflow + handles answer generation
    - DSPy Modules: Handle heavy reasoning for critical tasks (View Selection, SQL Generation)
    
    Workflow Steps:
    1. View Selection (IntegratedText2SQLModule - DSPy Heavy Reasoning)
    2. Schema Retrieval (get_entity_schema - Lightweight)
    3. SQL Generation (SQLWriterModule - DSPy Heavy Reasoning)
    4. Query Execution (execute_sql_query - Lightweight)
    5. Answer Generation (Agent Framework - Fast)
    """
    print("=" * 80)
    print("🚀 Complete Text-to-SQL Workflow with Agent Framework + DSPy")
    print("=" * 80)
    
    async with (
        AzureCliCredential() as credential,
        AzureAIClient(credential=credential).create_agent(
            name="CompleteText2SQLAgent",
            instructions='''You are an expert end-to-end Text-to-SQL agent for Snowflake databases.

Your complete workflow:

**STEP 1: View Selection & Entity Resolution**
- Call select_views_with_entities(question) to:
  - Resolve business entities (e.g., 'PE' → 'Private Equity', 'GCC' → Gulf countries)
  - Select relevant database views using Chain-of-Thought reasoning
- Display the resolved entities and selected views

**STEP 2: Schema Retrieval** (Fast Lookup)
- For each selected view, call get_entity_schema(view_name) to get:
  - Column names and types
  - Column definitions
  - Allowed values for categorical fields
- Show the key columns that will be used

**STEP 3: SQL Generation** 
- Call generate_sql_query(question, view_schemas, resolved_entities) to:
  - Generate optimized Snowflake SQL using DSPy Chain-of-Thought
  - Apply proper filters based on resolved entities
  - Use exact column names from schema
- Display the generated SQL query

**STEP 4: Query Execution** (Database Call)
- Call execute_sql_query(sql_query) to run the query
- Show the results in a clear format

**STEP 5: Answer Generation** (You handle this directly - Fast Response)
- Call generate_final_answer(question, sql_query, results, entities) to get context
- Based on the returned context and results, provide a clear business-friendly answer
- Include:
  * Direct answer with specific numbers
  * Context about what the data represents
  * Any relevant insights or observations
- Keep it concise and professional

**Important:**
- Show your reasoning at each step
- Display intermediate outputs for transparency
- If any step fails, explain why and suggest next steps
- Use DSPy tools (Steps 1 & 3) for complex reasoning where accuracy is critical
- Handle final answer generation yourself for faster response''',
            tools=[
                select_views_with_entities,
                get_entity_schema,
                generate_sql_query,
                execute_sql_query,
                generate_final_answer
            ],
        ) as agent,
    ):
        query = "What is MIC's current exposure in the USA in Private Equity?"
        print(f"\n👤 User Question: {query}")
        print("\n🤖 Agent Working...\n")
        print("-" * 80)
        
        async for chunk in agent.run_stream(query):
            if chunk.text:
                print(chunk.text, end="", flush=True)
        
        print("\n" + "=" * 80)
        print("✅ Workflow Complete!")
        print("=" * 80)


print("✅ Complete workflow function defined")
print("\nRun: await complete_text2sql_workflow()")


In [ ]:
# Run the complete workflow
await complete_text2sql_workflow()

## Architecture Summary

```
┌─────────────────────────────────────────────────────────────────────┐
│         Microsoft Agent Framework (Orchestration Layer)             │
│  • Multi-step workflow coordination                                 │
│  • Tool invocation sequencing                                       │
│  • Conversation management                                          │
│  • Streaming responses                                              │
│  • Final answer generation (FAST - no DSPy overhead)                │
└────────────────────────┬────────────────────────────────────────────┘
                         │
                         │ Orchestrates 5 Tasks
                         │
        ┌────────────────┴────────────────┐
        │                                  │
        ▼                                  ▼
┌──────────────────┐            ┌──────────────────────┐
│  DSPy Modules    │            │  Utility Functions   │
│ (Heavy Reasoning)│            │  (Lightweight)       │
└──────────────────┘            └──────────────────────┘
        │                                  │
        │                                  │
        ├─ Task 1: IntegratedText2SQL     ├─ Task 2: get_entity_schema
        │   • Entity Resolution (CoT)     │   • Schema retrieval
        │   • View Selection (CoT)        │   • Fast lookup
        │   • Time: ~3s                   │   • Time: <0.1s
        │   • Critical accuracy           │
        │                                 ├─ Task 4: execute_sql_query
        ├─ Task 3: SQL Generation         │   • Query execution
        │   • SQLWriterModule (CoT)       │   • Database call
        │   • Time: ~2-3s                 │   • Time: 1-10s
        │   • Critical accuracy           │
        │                                 └─ Task 5: generate_final_answer
        │                                     • Context preparation
        │                                     • Agent interprets (FAST!)
        │                                     • Time: ~1s
```

### Optimized Performance Profile:

| Task | Component | Reasoning Complexity | Time | Optimizable |
|------|-----------|---------------------|------|-------------|
| 1. View Selection | DSPy | **High** - Entity resolution + multi-dimensional matching | ~3s | ✅ GEPA/Bootstrap |
| 2. Schema Retrieval | Utility | Low - Direct lookup | <0.1s | ❌ Already fast |
| 3. SQL Generation | DSPy | **High** - Schema mapping + filter logic | ~2-3s | ✅ Bootstrap |
| 4. Query Execution | Utility | None - Database call | 1-10s | ❌ DB performance |
| 5. Answer Generation | **Agent** | Low - Interpretation | ~1s | ❌ Fast enough |

**Total Time: ~7-17 seconds** (acceptable for production)

### Key Benefits:

1. **Optimized Performance** ⚡
   - DSPy only for critical reasoning tasks (Steps 1 & 3)
   - Agent handles simple interpretation (Step 5) - Fast!
   - 2-3 seconds saved on final answer generation

2. **Strategic Reasoning Allocation** 🎯
   - **Heavy DSPy reasoning**: Where accuracy is critical (View Selection, SQL Generation)
   - **Lightweight Agent**: Where speed matters more (Answer formatting)

3. **Optimizable Components** 🔄
   - IntegratedText2SQL: Optimize with GEPA (15-20% accuracy gain)
   - SQLWriterModule: Optimize with Bootstrap (10-15% accuracy gain)
   - AnswerGenerator: Fast enough without optimization

4. **Transparency** 📋
   - Complete reasoning trace from DSPy modules
   - Agent provides natural language flow
   - Easy to debug and explain

5. **Production Ready** 🚀
   - Best tool for each job
   - Balanced speed vs. accuracy
   - Scalable architecture

## Comparison: Single Agent vs Multi-Agent Architecture

| Aspect | Single Agent + Tools | Multi-Agent Architecture |
|--------|---------------------|-------------------------|
| **Structure** | One agent with 5 tools | 4 specialized agents + orchestrator |
| **Complexity** | Simpler setup | More complex coordination |
| **Reasoning** | Single context window | Distributed reasoning across agents |
| **Debugging** | Harder to isolate issues | Easier - test agents independently |
| **Scalability** | Limited by context size | Each agent has fresh context |
| **Parallelization** | Sequential tool calls | Potential for parallel agent execution |
| **Error Handling** | One failure affects all | Isolated failures, easier recovery |
| **Cost** | Lower (1 agent session) | Higher (multiple agent sessions) |
| **Best For** | Simple workflows, cost-sensitive | Complex workflows, modularity needed |

### When to Use Each:

**Single Agent + Tools** (like `complete_text2sql_workflow`):
- ✅ Simpler workflows with clear sequential steps
- ✅ Cost-sensitive applications
- ✅ When all tasks fit in one context window
- ✅ Fast prototyping and testing

**Multi-Agent Architecture** (like `multi_agent_text2sql_workflow`):
- ✅ Complex workflows with many specialized tasks
- ✅ When different tasks require different expertise
- ✅ Need for parallel execution
- ✅ Better error isolation and testing
- ✅ Larger applications with team specialization

### Advanced Multi-Agent Patterns:

1. **Hierarchical**: Supervisor agent delegates to worker agents
2. **Sequential**: Agents pass work in a pipeline
3. **Parallel**: Multiple agents work simultaneously, results merged
4. **Collaborative**: Agents discuss and reach consensus
5. **Competitive**: Multiple agents propose solutions, best one selected

For your Text-to-SQL use case, **single agent + tools is recommended** for production due to:
- Lower cost (one agent session)
- Sufficient for sequential workflow
- Easier to maintain and deploy
- Faster execution (no agent handoff overhead)

Use multi-agent when you need to scale to handle dozens of different database types or complex query planning scenarios.

In [ ]:
# Run the multi-agent workflow
await multi_agent_text2sql_workflow()


In [ ]:
# ============================================================================
# Multi-Agent Architecture Implementation (Refactored for Clarity)
# ============================================================================

async def multi_agent_text2sql_workflow() -> None:
    """
    Text-to-SQL using Multi-Agent Architecture Pattern.
    
    Architecture:
    - View Selector Agent: Handles entity resolution + view selection (DSPy)
    - Schema Expert Agent: Retrieves and explains database schemas
    - SQL Generator Agent: Generates SQL queries (DSPy)
    - Data Analyst Agent: Interprets results and generates answers
    
    The main function orchestrates communication between specialized agents.
    """
    print("=" * 80)
    print("🤖 Multi-Agent Text-to-SQL Workflow")
    print("=" * 80)
    
    query = "What is MIC's current exposure in the USA in Private Equity?"
    print(f"\n👤 User Question: {query}\n")
    print("=" * 80)
    
    # Initialize Azure credentials (shared across all agents)
    async with AzureCliCredential() as credential:
        client = AzureAIClient(credential=credential)
        
        # ====================================================================
        # STEP 1: View Selection Agent
        # ====================================================================
        print("\n🤖 STEP 1: [View Selector Agent] Resolving entities and selecting views...")
        print("-" * 80)
        
        view_selection_result = ""
        async with client.create_agent(
            name="ViewSelectorAgent",
            instructions='''You are a specialized agent for database view selection.

Your expertise:
- Resolve business entities (e.g., 'PE' → 'Private Equity', 'MIC' → company identifier)
- Select optimal database views for answering questions
- Use Chain-of-Thought reasoning via DSPy module

When you receive a question:
1. Call select_views_with_entities(question) 
2. Return a structured response with:
   - Resolved entities (JSON format)
   - Selected view names (list)
   - Reasoning for your selections

Be concise and precise.''',
            tools=[select_views_with_entities],
        ) as view_agent:
            async for chunk in view_agent.run_stream(query):
                if chunk.text:
                    print(chunk.text, end="", flush=True)
                    view_selection_result += chunk.text
        print("\n")
        
        # ====================================================================
        # STEP 2: Schema Expert Agent
        # ====================================================================
        print("\n🤖 STEP 2: [Schema Expert Agent] Retrieving database schemas...")
        print("-" * 80)
        
        schema_query = f"Get schemas for the views needed to answer: {query}\n\nContext from View Selector:\n{view_selection_result}"
        schema_info = ""
        
        async with client.create_agent(
            name="SchemaExpertAgent",
            instructions='''You are a database schema expert.

Your role:
- Retrieve detailed schema information for database views
- Explain column meanings, types, and relationships
- Identify key columns relevant to the question

When given view names:
1. Call get_entity_schema(view_name) for each view
2. Summarize the most relevant columns
3. Highlight any important constraints or allowed values

Format your response clearly for the SQL Generator.''',
            tools=[get_entity_schema],
        ) as schema_agent:
            async for chunk in schema_agent.run_stream(schema_query):
                if chunk.text:
                    print(chunk.text, end="", flush=True)
                    schema_info += chunk.text
        print("\n")
        
        # ====================================================================
        # STEP 3: SQL Generator Agent
        # ====================================================================
        print("\n🤖 STEP 3: [SQL Generator Agent] Generating SQL query...")
        print("-" * 80)
        
        sql_query_prompt = f"""Generate SQL for: {query}

View Selection Context:
{view_selection_result}

Schema Information:
{schema_info}

Generate the SQL query now."""
        sql_result = ""
        
        async with client.create_agent(
            name="SQLGeneratorAgent",
            instructions='''You are an expert SQL query generator for Snowflake.

Your expertise:
- Generate syntactically correct Snowflake SQL
- Use Chain-of-Thought reasoning via DSPy module
- Apply filters based on resolved entities

When you receive:
- Original question
- View schemas with column details
- Resolved entities

Call generate_sql_query(question, view_schemas, resolved_entities) to create optimized SQL.
Return the SQL query with your reasoning.''',
            tools=[generate_sql_query],
        ) as sql_agent:
            async for chunk in sql_agent.run_stream(sql_query_prompt):
                if chunk.text:
                    print(chunk.text, end="", flush=True)
                    sql_result += chunk.text
        print("\n")
        
        # ====================================================================
        # STEP 4: Query Execution (Utility - not an agent)
        # ====================================================================
        print("\n⚙️  STEP 4: [Query Execution] Running SQL...")
        print("-" * 80)
        
        # Mock execution for demo - replace with actual Snowflake connection
        mock_results = """
Query Results:
┌──────────┬─────────────────┬───────────────────┐
│ PLATFORM │ COUNTRY         │ UNREALIZED_VALUE  │
├──────────┼─────────────────┼───────────────────┤
│ PE       │ USA             │ $2,450,000,000    │
└──────────┴─────────────────┴───────────────────┘

(1 row returned)
"""
        print(mock_results)
        
        # ====================================================================
        # STEP 5: Data Analyst Agent
        # ====================================================================
        print("\n🤖 STEP 5: [Data Analyst Agent] Interpreting results...")
        print("-" * 80)
        
        analyst_prompt = f"""Analyze these results and answer the user's question.

Original Question: {query}

SQL Query Context:
{sql_result}

Query Results:
{mock_results}

Provide a clear business answer."""
        
        async with client.create_agent(
            name="DataAnalystAgent",
            instructions='''You are a business data analyst.

Your role:
- Interpret SQL query results
- Provide clear, business-friendly answers
- Include context and insights

When you receive query results:
1. Call generate_final_answer(question, sql_query, results, entities)
2. Provide a concise answer with:
   - Direct answer to the question
   - Specific numbers from results
   - Business context
   - Any relevant observations

Be professional and concise.''',
            tools=[generate_final_answer],
        ) as analyst_agent:
            async for chunk in analyst_agent.run_stream(analyst_prompt):
                if chunk.text:
                    print(chunk.text, end="", flush=True)
        print("\n")
        
        print("\n" + "=" * 80)
        print("✅ Multi-Agent Workflow Complete!")
        print("=" * 80)


print("✅ Multi-agent workflow function defined (refactored for clarity)")
print("\nRun: await multi_agent_text2sql_workflow()")


## Multi-Agent Architecture Pattern

In this implementation, the **orchestrator is the Python function itself** (`multi_agent_text2sql_workflow`), not a separate agent. It coordinates the sequential workflow:

```
┌─────────────────────────────────────────────────────────────────────┐
│         Python Function (Orchestrator Logic)                        │
│         multi_agent_text2sql_workflow()                             │
│  • Receives user question                                           │
│  • Creates agents sequentially                                      │
│  • Passes context between agents                                    │
│  • Manages workflow state                                           │
└────────────────────────┬────────────────────────────────────────────┘
                         │
                         │ Creates & Coordinates Agents
                         │ (Sequential Pipeline)
                         │
        ┌────────────────┴────────────────────────────────┐
        │                │                │                │
        ▼                ▼                ▼                ▼
┌─────────────┐  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐
│View Selector│  │Schema Expert│  │SQL Generator│  │Data Analyst │
│   Agent     │  │   Agent     │  │   Agent     │  │   Agent     │
│  (Step 1)   │  │  (Step 2)   │  │  (Step 3)   │  │  (Step 5)   │
└─────────────┘  └─────────────┘  └─────────────┘  └─────────────┘
        │                │                │                │
        │                │                │                │
   Uses DSPy      Fast lookup      Uses DSPy       Interprets
IntegratedT2SQL                  SQLWriter         results
        │                │                │                │
        └────────────────┴────────┬───────┴────────────────┘
                                  │
                              Step 4: Query Execution
                              (Utility function)
```

**Key Differences from Traditional Multi-Agent Systems:**

1. **Orchestrator = Python Function**: 
   - Not a separate LLM agent
   - Deterministic workflow logic
   - Explicit control flow

2. **Sequential Handoffs**:
   - Each agent completes its task
   - Output passed as input to next agent
   - State managed by Python variables

3. **Agent Lifecycle**:
   - Agents created on-demand
   - Used for single task
   - Cleaned up after completion

**Benefits:**
1. **Predictable Flow**: Python function ensures deterministic execution order
2. **Error Isolation**: Each agent's failure is contained in its step
3. **Easy Testing**: Test individual agents or the entire workflow
4. **Clear Debugging**: Print statements show exact workflow progress
5. **Cost Efficient**: Agents only exist when needed

**Alternative: True Orchestrator Agent**

For dynamic workflows where the sequence isn't predetermined, you could create an actual orchestrator agent that decides which specialized agent to call based on the conversation state.

## ⚠️ Important: Double LLM Calls in Multi-Agent Architecture

You've identified a **critical inefficiency** in the multi-agent approach:

### The Double LLM Problem:

**In Multi-Agent Architecture (Approach 2):**
```
User Question
    ↓
View Selector Agent (MAF - LLM Call #1)
    ↓ calls tool
select_views_with_entities() → DSPy Module (LLM Call #2)
    ↓
Result back to agent
```

**Each specialized agent makes TWO LLM calls:**
1. **Agent LLM call**: Agent reasoning to decide which tool to call
2. **DSPy LLM call**: DSPy module performing the actual heavy reasoning

**In Single Agent Architecture (Approach 1):**
```
User Question
    ↓
Single Agent (MAF - LLM Call #1)
    ↓ calls tool
select_views_with_entities() → DSPy Module (LLM Call #2)
    ↓
Result back to same agent
```

### Cost & Latency Comparison:

| Architecture | LLM Calls per Task | Total LLM Calls | Agent Overhead |
|--------------|-------------------|-----------------|----------------|
| **Single Agent + Tools** | 2 (agent + DSPy) | ~10 calls | Low - reuses context |
| **Multi-Agent** | 2 per agent × 4 agents | ~16+ calls | High - fresh context each time |

### Why Approach 1 (Single Agent) is Better:

✅ **Fewer LLM Calls**: One agent context across all steps  
✅ **Lower Cost**: ~40% fewer LLM invocations  
✅ **Faster**: No agent-to-agent handoff overhead  
✅ **Shared Context**: Agent maintains conversation history  
✅ **Simpler Debugging**: Single agent conversation to trace  

### When Multi-Agent Might Make Sense:

Only use multi-agent if you need:
- **Different LLM models** per task (e.g., GPT-4o for view selection, GPT-4o-mini for schema lookup)
- **True parallelization** of independent tasks
- **Agent specialization** with different prompt templates and temperature settings
- **Fault isolation** where agent failures shouldn't affect others

### Recommendation for Your Use Case:

**Use Single Agent + Tools (Approach 1)** because:
1. Your workflow is strictly sequential (can't parallelize)
2. All tasks use the same LLM model (GPT-4o)
3. Tasks need shared context (entities, views, schemas)
4. Cost and speed matter more than modularity

The multi-agent pattern is educational but not optimal for this specific Text-to-SQL pipeline.